# Homework 4: Forwards and Backwards

## Introduction and Theoretical Explanation

In this assignment we'll be implementing a basic fully-connected feed-forward neural network with batch normalization using numpy. We will then be using it to classify characters in the MNIST dataset. 

By fully-connected we mean that the output for each unit in the prior layer is given as input for each unit in the following layer.

By feed-forward we mean that the model has a series of layers in which the output of one layer, either fully or partially, is used as input for a subsequent layer, but not a prior layer.

<div style="text-align: center;">
    <img src="nn-diagram.png" alt="fdafdas" width="400"/>
     <p style="text-align: center;">
        Source: Gow, Stephen & Niranjan, Mahesan & Pearman-Kanza, Samantha & Frey, Jeremy. (2022). 
        A Review of Reinforcement Learning in Chemistry.<em> Digital Discovery. 1.</em> 10.1039/D2DD00047D.
    </p>
</div>

Above is a diagram of a very simple fully-connected feed-forward neural network. Each layer in the diagram are represented by a column of dots, which represent units. 

Mathematically we can think of these networks as a series of composed functions, expressed as:

$$ o = f_3(f_2(f_1(x))) $$

where
- $x$ is the input
- $f_i$ is the $i^{th}$ fully-connected layer
- $o$ is the output of the network

Note that the above diagram has three layers in total, two hidden layers and an output layer. Each layer has multiple units which receive input and send their output to the following layer. Each unit is comprised of two components: a linear regression function and an activation function. In mathematical notation we could state this as:

$$ o = a(w_1x_1 + w_2x_2 + ... + w_nx_n + b) $$

where
- $o$ is the output of the unit
- $a$ is the activation function, a non-linear function that we discuss more later in the homework
- $w_i$ is the $i^{th}$ weight of the linear regression function
- $x_i$ is the $i^{th}$ component of the input
- $b$ is the bias term


or rather, expressing the output of the whole layer,

$$ o^{(i)} = a(W^{(i)}x^{(i)} + b^{(i)}) $$
where 
- $o^{(i)}$ is the output of the layer, a one dimensional vector with the same length as the number of units in the layer
- $x^{(i)}$ is the input into the $i^{th}$ layer, which is often equal to the output of the previous layer $o^{(i-1)}$
- $W^{(i)}$ is a matrix with dimensions (num_units, input_size) that holds the weights of all regression units in the layer
- $b^{(i)}$ is the bias term which is a one dimensional vector with the same length as the number of units in the layer

Let's combine these formulations to express the above network in terms of functional composition. The components of a given layer are shown in the same color.

$$
o = \color{red}a^{(3)}(W^{(3)}\color{blue}a^{(2)}(W^{(2)}\color{green}a^{(1)}(W^{(1)}(\color{black}x\color{green}) + b^{(1)})\color{blue} + b^{(2)}) \color{red} + b^{(3)})
$$

From this you should have an idea of how an input is evaluated by a neural network of the sort we're implementing today. In this homework we start by writing classes for the components of the network and then combining them at the end when writing the `Model` class. Each of the classes we'll be implementing today will have a `forward` method which will do part of the above computation for the respective component.

Sadly, neural networks are not magic, and they are not useful freshly initialized with random values. To make them useful and accurately classifying we have to adjust the weights and biases in each unit. To do this we'll use a method called mini-batch training that finds which way and how much we need to adjust each weight and bias in the network. This technique requires a loss function, often denoted with $\mathcal{L}$, which tells us how wrong our predictions are. 

To figure out how to adjust each weight we'll need to calculate the gradient of the weight or bias with respect to $\mathcal{L}$, or in mathematical notation, where $z$ is the output of the linear regression:

$$ \frac{\partial\mathcal{L}}{\partial w} = \frac{\partial\mathcal{L}}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot \frac{\partial z}{\partial w} \text{  or  } \frac{\partial\mathcal{L}}{\partial b} = \frac{\partial\mathcal{L}}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot \frac{\partial z}{\partial b}$$

To calculate the gradient of the weights we'll utilize the chain rule of derivation:

$$ \frac{d}{dx}f(g(h(x))) = f'(g(h(x))) \cdot g'(h(x)) \cdot h'(x) $$

an when calculating the gradient of the activation function we'll use the additive rule of derivation as well:

$$ \frac{d}{dx}(f(x) + g(x) + h(x)) = f'(x) + g'(x) + h'(x) $$

With this, we'll only need to calculate the gradient of a particular function in the network, and pass this to a prior layer so that it can calculate its own gradient. The specifics of this will be discussed below, as each component of the network has its own `backward` method that computes such a gradient.

In [2]:
import numpy as np
from typing import Any, List, Optional

## Linear class

The linear layer contains multiple linear regression units. The input is equal to the output size of the previous layer, or rather the number of linear regression units in the previous layer, or the input size, if the linear layer is the first layer in the network. A given linear layer has an output size that is equal to the number of linear regression units in the layer, as each linear regression unit outputs only one value.

Below you will implement the `__init__`, `forward`, `backward`, and `update_parameters` methods of the layer. Please note that for a given call to `forward` or `backward` you might receive multiple inputs which are stacked together in a two dimensional array. In this case, the shape of the input will be `(num_inputs, input_size)`.

The `__init__` method initializes the layer. The layer should contain the following at minimum:

- A two dimensional numpy array that contains the weights of all linear regression units in the layer. The initial values of these weights should be initialized to random values from a Gaussian distribution with $\mu = 0$ and $\sigma = 1$, (Hint: `randn`) and scaled (or rather multiplied) by $ \sqrt{\frac{2}{\mathrm{input\_size}}} $.
  
- A one dimensional numpy array that contains the biases for each linear regression unit which are initialized to zero.

- _Optional Parameters_: If you want to be explicit about the other values that will later be saved within the object, you can declare them in the `__init__` method and set them to `None`. This would include the gradients of the weights, the gradients of the biases, and the previous input, all of which are used during the training.

The `forward` method computes the regression against the inputs. The formula for an individual regression unit is as such:
$$
w_{1j}x_1 + w_{2j}x_2 + ... + w_{nj}x_n + b_j
$$ 
where $n$ is the input size, $w_{ij}$ is the weight for input entry $i$ in regression unit $j$, and $b_j$ is the bias term for regression unit $j$. To do this in parallel, I would suggest using matrix multiplication. You should save the input from the forward pass, as the input is required when calculating the gradient of the backward pass.  

The `backward` method computes the gradients of the weights and biases given the gradient of the loss function with respect to the outputs and saves them as an internal variable. The gradients of the weights and biases should be the same size as the array that holds the weights and biases, respectively. This method returns the gradient of the input, which likewise will be the size of the input.

Since we are given the gradient of the output as a parameter, we just need to compute the gradient of a given weight with respect to the output. Let $i$ be the index of the weight, or rather the entry in the input, and $j$ be the index of the unit, and $k$ be the index of a data point in the complete input, since remember, we should be able to pass multiple inputs in one call.

$$
\frac{\partial \mathcal{L}}{\partial w_{ij}} = \frac{\partial \mathcal{L}}{\partial z_j} \cdot \frac{\partial z_j}{\partial w_{ij}} = \frac{\partial \mathcal{L}}{\partial z_j} * x_i \text{ or } \sum_k  \frac{\partial \mathcal{L}}{\partial z_{jk}} * x_{ik} 
$$

$$
\frac{\partial \mathcal{L}}{\partial b_j} = \frac{\partial \mathcal{L}}{\partial z_j} \cdot \frac{\partial z_j}{\partial b_j} = \frac{\partial \mathcal{L}}{\partial z_j} \text{ or } \sum_k \frac{\partial \mathcal{L}}{\partial z_{jk}}
$$

$$
\frac{\partial \mathcal{L}}{\partial x_{ik}} = \sum_j \frac{\partial \mathcal{L}}{\partial z_j} \cdot \frac{\partial z_j}{\partial x_{ik}} = \sum_j \frac{\partial \mathcal{L}}{\partial z_j} * w_{ij} 
$$

All of this can be done via matrix multiplications.

The `update_parameters` method updates the weights and biases of the network. The formula for this is as follows:
$$
w_{ij} \leftarrow w_{ij} - \eta \frac{\partial \mathcal{L}}{\partial w_{ij}}
$$
and likewise for the bias:
$$
b_{j} \leftarrow b_{j} - \eta \frac{\partial \mathcal{L}}{\partial b_{j}}
$$
where $\eta$ is the learning rate.

In [ ]:
class Linear:
    """Implements a linear (fully connected) layer."""

    def __init__(self, input_dim: int, output_dim: int) -> None:
        """
        Initialize the layer with input and output dimensions.

        Args:
            input_dim (int): The size of each input sample.
            output_dim (int): The size of each output sample.
        """
        self.weights = np.random.normal(0, 1, (input_dim, output_dim))
        self.bias = np.zeros(input_dim)
        self.grads = None

    def __call__(self, x: np.ndarray) -> np.ndarray:
        """
        Make the layer callable.

        Args:
            x (np.ndarray): Input data.

        Returns:
            np.ndarray: Output of the linear layer.
        """
        return self.forward(x)

    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the forward pass of the linear layer.

        Args:
            x (np.ndarray): Input data.

        Returns:
            np.ndarray: The linear transformation of the input.
        """
        return x * self.weights * self.bias

    def backward(self, grad_output: np.ndarray) -> np.ndarray:
        """
        Compute the backward pass of the linear layer.

        Args:
            grad_output (np.ndarray): Gradient of the loss with respect to the output.

        Returns:
            np.ndarray: Gradient of the loss with respect to the input.
        """
        return np.gradient(grad_output) * np.gradient(self.weights)

    def update_params(self, learning_rate: float) -> None:
        """
        Update the weights and biases using the computed gradients.

        Args:
            learning_rate (float): The learning rate for parameter updates.
        """
        self.weights = self.weights - learning_rate * self.backward(self.grads)



## Activation function: ReLU

Between each layer of a neural network you'll find a non-linear function. This is the key feature that allows a neural network to approximate non-linear functions. If we were to not have a non-linear function between layers, we'd have a linear model, and all multi-layer linear models can be reduced to a single linear regression calculation (I'll leave the proof as a fun exercise that has no applicability to the course). 

A rectified linear unit (ReLU) is a common non-linear function found in neural networks. In this context, the non-linear function is also called a non-linearity or an activation function. Other activation functions include hyperbolic tangent, sigmoid, and exponential linear unit, among many others, but they are all out of scope for this exercise.

In the below class we'll be implementing the forward and backward passes of the ReLU function. 

The forward pass implements the function and applies it to all entries in the input array. The ReLU  for layer $i$ is defined as follows:

$$ a^{(i)} = \mathrm{ReLU}(x) = \mathrm{max}(0, x) $$

The backward pass implements the partial derivative of the ReLU function, which is defined as such:

$$
\frac{\partial \mathrm{ReLU(x)}}{\partial x} = 
\begin{cases}
    1 & \text{if } x > 0, \\
    0 & \text{if } x \leq 0
\end{cases}
$$

During training the backward pass requires an input that the gradient of the output is multiplied with, so the resulting computation would be:

$$
\frac{\partial \mathcal{L}}{\partial z^{(i)}} = \frac{\partial \mathcal{L}}{\partial a^{(i)}} \cdot \frac{\partial a^{(i)}}{\partial z^{(i)}} 
$$

Hint: You might want to make a copy of `grad_output`.

In [ ]:
class ReLU:
    """Implements the ReLU activation function."""

    def __init__(self) -> None:
        """Initialize the ReLU activation function."""
        pass

    def __call__(self, x: np.ndarray) -> np.ndarray:
        """
        Make the ReLU activation callable.

        Args:
            x (np.ndarray): Input data.

        Returns:
            np.ndarray: Output after applying ReLU.
        """
        return self.forward(x)

    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the forward pass of ReLU activation.

        Args:
            x (np.ndarray): Input data.

        Returns:
            np.ndarray: Output after applying ReLU.
        """

### Please enter your solution here ###


    def backward(self, grad_output: np.ndarray) -> np.ndarray:
        """
        Compute the backward pass of ReLU activation.

        Args:
            grad_output (np.ndarray): Gradient of the loss with respect to the output.

        Returns:
            np.ndarray: Gradient of the loss with respect to the input.
        """

### Please enter your solution here ###



## Loss function: Cross Entropy

The loss function is a measure of how far off the results of the network are from the actual results. We can think of the label of the sample $y$ as a sample from a probability distribution $p(x)$, however one where all samples have converged on a particular label. Likewise, we can think of the network output $\hat y$ as a sample of $q(x)$.

In our case our labels are integers in [0,9], however they can be transformed into a one-hot encoding, where one entry is 1 and the rest 0, representing the classification of a particular category which that index represents. In our case, these categories can be directly represented by the first 10 indices, coincidentally in a very literal way, as our categories are the digits 0 through 9, but such encoding can also be used for arbitrary exclusive categories such as "hotdog" and "not hotdog" or "cat", "dog", and "other input".

For example:

`3 = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`

In this way, our labels can be accurately compared with the output of the network, which is also a one-dimensional array of length 10. We can think of these labels as the probability of the input being a member of a certain category. For known data, the probability distribution has converged to the label, however, when the network makes a prediction, we will not see such absolute convergence.

One problem we have is that the output of the network might not sum to 1 and all of the values might not be in $[0,1]$, as is a requirement for a probability distribution. The raw network outputs will almost never have this property, however we can force this property by normalizing the output. To do this we'll apply a **softmax** function to the output of the network. For a the $i^{th}$ entry is expressed as such after a softmax is applied:

$$
q_i = \frac{e^{\hat y_i}}{\sum_{j} e^{\hat y_j}}
$$

Since $e^x$ is monotonically increasing, or rather only increases as $x$ increases, we still have the same relative ordering of entries in the output, although the relative difference between the entries has changed. This relative ordering is important, as it will still allow us to find the most likely category by using the largest value in the output without applying the softmax during inference. Unfortunately, for reasons we won't go into, this formulation is not very numerically stable, and instead we'll calculate the softmax so that all $z$ values are subtracted by the largest entry in the output so that maximum value in the numerator of the new output is 1:

$$
q_i = \frac{e^{\hat y_i - \max_{j} \hat y_j}}{\sum_{k} e^{\hat y_k - \max_{j} \hat y_j}} = \frac{e^{(-\max_{j} \hat y_j)}e^{\hat y_i}}{\sum_{k}e^{(-\max_{j} \hat y_j)}e^{\hat y_k}} = \frac{e^{(-\max_{j} \hat y_j)}e^{\hat y_i}}{e^{(-\max_{j} \hat y_j)}\sum_{k}e^{\hat y_k}} = \frac{e^{\hat y_i}}{\sum_{k}e^{\hat y_k}}
$$

As we see, this adjustment is equivalent to the original softmax, but is more CPU friendly.

The loss function $\mathcal{L}(p, q)$ we'll be using is called a **Cross Entropy Loss**, which measures the difference between two probability distributions. Since our labels and our network outputs can be interpreted as this, we can use this. The loss for a single sample is expressed as such:

$$
\mathcal{L}(p, q) = -\sum^{K}_{k=1} p_k \log( q_k)
$$

where 
- $p$ is the true label with $p_k$ being the $k^{th}$ element of $p$.
- $q$ is the network output with the softmax applied, with $q_k$ being the $k^{th}$ element of $q$.
- And $K$ is the number of elements in $p$ and $q$

When applying the loss function to multiple samples, we simply average all the individual losses of the samples:

$$
\text{Loss} = -\frac{1}{N}\sum^{N}_{n=1}\sum^{K}_{k=1} p_{n,k} \log( q_{n,k})
$$

where
- $N$ is the number of samples being calculated in the loss function.
- $p_{n,k}$ is the $k^{th}$ element of the $n^{th}$ sample label.
- $q_{n,k}$ is the $k^{th}$ element of the $n^{th}$ network output with softmax.

It is important to note that during that `backward` pass we will not be using this average, but rather the losses of all individual samples. The average loss, however, is an important metric to keep track of while developing a machine learning model.

We can combine and simplify the softmax output and the cross entropy loss. First note that in $\mathcal{L}(p, q)$, the only entry in the sum that does not evaluate to 0 is the entry where $p_k = 1$, let's call the index where this is the case $y$. Thus we can say:

$$
\mathcal{L}(p, q) = - \log(q_y)
$$

Combining this with the cross entropy loss we get:

$$
\mathcal{L}(p, q) = -\log\left( \frac{e^{\hat y_y - \max_{j} \hat y_j}}{\sum_{k} e^{\hat y_k - \max_{j} \hat y_j}} \right) = - (\hat y_y - \max_{j}(\hat y_j)) + \log \left( \sum^K_{k=1} e^{\hat y_k - \max_{j}(\hat y_j)}\right)
$$

Writing this as the loss function for multiple samples we get:

$$
\text{Loss} = - \frac{1}{N} \sum^N_{n=1}  \log\left( \frac{e^{\hat y_{n,y} - \max_{j} \hat y_{n,j}}}{\sum_{k} e^{\hat y_{n,k} - \max_{j} \hat y_{n,j}}} \right) = \frac{1}{N} \sum^N_{n=1} \left( - (\hat y_{n,y} - \max_{j}(\hat y_{n,j})) + \log \left( \sum^K_{k=1} e^{\hat y_{n,k} - \max_{j}(\hat y_{n,j})}\right) \right)
$$

The above formulation is what should be implemented in the `forward` function. Be sure to save the probabilities from the softmax for use in the `backward` method.

The purpose of the `backward` method is to calculate the gradient of $\mathcal{L}(p, q)$ with respect to the network's outputs, or in mathematical notation:

$$
\frac{\partial \mathcal{L}}{\partial z_i} = \frac{\partial \mathcal{L}}{\partial q_i} \cdot \frac{\partial q_i}{\partial z_i} + \sum_{i \neq j} \frac{\partial \mathcal{L}}{\partial q_i} \cdot \frac{\partial q_i}{\partial z_j}
$$

Since $\mathcal{L} = - \log(q_y)$ we can easily fine the gradient of $\mathcal{L}$ with respect to $q_i$.

$$
\frac{\partial \mathcal{L}}{\partial q_i} =
\begin{cases}
    -\frac{1}{q_i} & \text{if } i = y \\
    0 & \text{if } i \neq y
\end{cases}
$$

The partial gradient of $q_j$ with respect to $z_i$, or rather the partial derivative of the softmax function, is as follows:

$$
\frac{\partial q_j}{\partial z_i} =
\begin{cases}
    q_j(1-q_j) & \text{if } i = j \\
    -q_j q_i & \text{if } i \neq j
\end{cases}
$$

Putting these together we'll consider the cases of the logit with the correct class ($i = y$) and the logits of the incorrect classes ($i \neq y$).

$$
\frac{\partial \mathcal{L}}{\partial z_i} = 
\begin{cases}
    -\frac{1}{q_y} \cdot q_y(1 - q_y) = -(1 - q_y) = q_y - 1 & \text{if } i = y \\
    -\frac{1}{q_y} \cdot (-q_i q_y) = q_i & \text{if } i \neq y
\end{cases}
$$

Putting this together, and considering what we know about one-hot encoding we can say:

$$
\frac{\partial \mathcal{L}}{\partial z_i} = q_i - y_i
$$

Since we're doing this over many samples, we'll want to adjust the gradient in proportion to the number of samples:

$$
\frac{\partial \mathcal{L}}{\partial z_i} = \frac{1}{N} ( q_{i} - y_{i} )
$$

This adjustment makes it so that the number of samples, or rather our batch size, doesn't affect the magnitude of our weight and bias gradients.

In [ ]:
class CrossEntropy:
    """Implements the cross-entropy loss function with softmax."""

    def __init__(self) -> None:
        """Initialize the CrossEntropy loss function."""

### Please enter your solution here ###


    def __call__(self, logits: np.ndarray, labels: np.ndarray) -> float:
        """
        Compute the cross-entropy loss.

        Args:
            logits (np.ndarray): Logits predicted by the model.
            labels (np.ndarray): True labels.

        Returns:
            float: The cross-entropy loss.
        """
        return self.forward(logits, labels)

    def forward(self, logits: np.ndarray, labels: np.ndarray) -> float:
        """
        Compute the forward pass of the loss function.

        Args:
            logits (np.ndarray): Logits predicted by the model.
            labels (np.ndarray): True labels.

        Returns:
            float: The cross-entropy loss.
        """

### Please enter your solution here ###


    def backward(self) -> np.ndarray:
        """
        Compute the backward pass of the loss function.

        Returns:
            np.ndarray: Gradient of the loss with respect to the logits.
        """

### Please enter your solution here ###



## Model class

The `Model` class integrates all of the classes we have written so far together, taking a list of layers and moving outputs forward and gradients backwards.

Implement the `__init__` class which takes a list of instantiated classes which contain, at the very least, the `forward` and `backward` methods, and saves it as an internal variable. This also includes the `ReLU` class, even though it's not often considered a layer on its own.

Implement the `forward` method which takes an input and passes it to the first layer, for any given layer, using the output of the previous layer as input for the given layers `forward` method. This is done until the `forward` method of the last layer has returned, which is the output of the network.

Implement the `backward` method, which takes the gradient of the loss function and use it as input for the `backward` method of the last layer. Any given output of a `backward` method is used as input for the `backward` method of the previous layer. Continue this until there is no previous layer.

Implement the `update_params` which calls the `update_params` method for every layer, assuming that the layer has a method `update_params`.

Training a neural network using mini-batches involves repeatedly using a random sampling of the training set to get network output (`forward`), finding how wrong the outputs were (`loss_fn`), propagating the gradient of the loss function up the network (`backward`), and updating all parameters in the network (`update_params`). One iteration of this process is called a **batch**. When using mini-batches we'll generally use 64 or 128 samples per batch, but the defining trait is that we're using more than one sample, but not the entire training set. We will do this for a certain number of **epochs**, or passes through the entire training set.

Every epoch we will want to keep track of the total loss of all samples, and the accuracy of the model given all samples. We will do this both for the training data and for the validation data. After every epoch we will want to evaluate the model on our validation set. The purpose of the the validation set is to see how well the network generalizes on data it hasn't been trained on. A model can do very well on the data that it is trained on, as it could have easily 'memorized' it, which is why it's important to see how well it performs on data it has never seen. 

Since our batch size will not be 1 we will have to adjust the total loss for each batch, as the reported loss from the loss functions `forward` method will only be the average. Practically, this means multiplying the reported loss of the batch by the batch size.

Before every epoch, be sure to shuffle the training data, as this will ensure that the samples the network receives are randomized, so that it the network doesn't get the same samples for a batch for multiple epochs, and so that no samples are used more than once per epoch.

`train` should return a dictionary with the keys `"train_acc"`, `"val_acc"`, `"train_loss"`, and `"val_loss"`, which contains the accuracy and loss of the model on the training and validation set for every epoch.

Below you will have to fill in the code for the `__init__` method, the `forward` method, the `backward` method, and the portions of the `train` method that shuffle the training set for the epoch, perform the procedure for a batch, and calculate the accuracy and loss of the validation set. 

In [ ]:

class Model:
    """Represents a feed-forward neural network model."""

    def __init__(self, layers: List[Any]) -> None:
        """
        Initialize the model with a list of layers.

        Args:
            layers (List[Any]): List of layers (e.g., Linear, ReLU).
        """

### Please enter your solution here ###


    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Perform a forward pass through all layers.

        Args:
            x (np.ndarray): Input data.

        Returns:
            np.ndarray: Output of the model.
        """

### Please enter your solution here ###


    def backward(self, grad_output: np.ndarray) -> None:
        """
        Perform a backward pass through all layers.

        Args:
            grad_output (np.ndarray): Gradient of the loss with respect to the output.
        """

### Please enter your solution here ###


    def update_params(self, learning_rate: float) -> None:
        """
        Update parameters of all layers.

        Args:
            learning_rate (float): Learning rate for parameter updates.
        """

### Please enter your solution here ###


    def train(self, loss_fn: CrossEntropy,
              X_train: np.ndarray, y_train: np.ndarray,
              X_val: np.ndarray, y_val: np.ndarray,
              batch_size: int, num_epochs: int, learning_rate: float) -> dict:
        """
        Train the model using stochastic gradient descent.

        Args:
            loss_fn (CrossEntropy): The loss function to use.
            X_train (np.ndarray): Training data inputs.
            y_train (np.ndarray): Training data labels.
            X_val (np.ndarray): Validation data inputs.
            y_val (np.ndarray): Validation data labels.
            batch_size (int): Size of each training batch.
            num_epochs (int): Number of epochs to train.
            learning_rate (float): Learning rate for parameter updates.

        Returns:
            dict: Dictionary containing 'train_loss', 'val_loss', 'train_acc', 'val_acc'.
        """
        num_samples: int = X_train.shape[0]
        history = {
            'train_loss': [],
            'val_loss': [],
            'train_acc': [],
            'val_acc': []
        }

        for epoch in range(num_epochs):
            # Shuffle the training data

### Please enter your solution here ###


            # Training
            epoch_loss = 0
            correct_preds = 0
            num_batches = 0

            for start_idx in range(0, num_samples, batch_size):

                # Select samples for batch
                # Forward pass
                # Compute loss
                # Compute training accuracy
                # Backward pass
                # Update parameters


### Please enter your solution here ###


                num_batches += 1

            # Compute average loss and accuracy over the epoch
            avg_train_loss = epoch_loss / num_samples
            train_accuracy = correct_preds / num_samples

            # Validation

### Please enter your solution here ###


            # Record history
            history['train_loss'].append(avg_train_loss)
            history['train_acc'].append(train_accuracy)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_accuracy)

            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {avg_train_loss:.6f}, Train Acc: {train_accuracy * 100:.2f}%, "
                  f"Val Loss: {val_loss:.6f}, Val Acc: {val_accuracy * 100:.2f}%")

        return history

# Putting things together

Below we import the MNIST dataset, define the network architecture, instantiate the model, and partition our dataset into train, validation, and test partitions. 

In [ ]:
from utils import get_mnist

X, y = get_mnist()

start_train_idx = 0
end_train_idx = 50000
start_val_idx = end_train_idx
end_val_idx = 60000
start_test_idx = end_val_idx
end_test_idx = 70000

X_train, y_train = X[start_train_idx:end_train_idx], y[start_train_idx:end_train_idx]
X_val, y_val = X[start_val_idx:end_val_idx], y[start_val_idx:end_val_idx]
X_test, y_test = X[start_test_idx:end_test_idx], y[start_test_idx:end_test_idx]

# Playing with the learning rate

The learning rate $\eta$ (found in `update_params` in the Linear class) globally regulates the degree to which we adjust our weights and bias terms. Too small learning rate might result in slow convergence or convergence to a local minimum that is sub-optimal, too large a learning rate might prevent the model from converging as it 'bounces' around a minimum. Thus, if we are to use mini-batch gradient descent as our learning method, it's important to have a learning rate that's just right. 

Below you will train three different models with three different learning rates across 20 epochs. 

Plot the validation accuracies for each learning rate along with the test accuracies as dashed horizontal lines. The colors of lines representing the same learning rate should be the same color. Show the legend in this plot and include proper axis labels and title in the plot. 

In [ ]:
batch_size = 64
num_epochs = 20
learning_rates = [0.01, 0.1, 1.0]

input_dim = X_train.shape[1] # 28x28 pixels
output_dim = 10 # Digits 0-9

histories= []
test_accuracies = []

for lr in learning_rates:
    print(f"\nTraining model with learning rate {lr}...")

    layers: List[Any] = [
        Linear(input_dim, 64),
        ReLU(),
        Linear(64, output_dim)
    ]

    loss_fn = CrossEntropy()

    model = Model(layers)
    
    # Train the model
    history = model.train(
        loss_fn=loss_fn,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        batch_size=batch_size,
        num_epochs=num_epochs,
        learning_rate=lr
    )
    histories.append((lr, history))

    # Evaluate the model on the test set

### Please enter your solution here ###

    print(f"Test Accuracy for learning rate {lr}: {test_accuracy * 100:.2f}%")

In [ ]:
# Plotting Code
import matplotlib.pyplot as plt

def plot_val_accuracy_history(histories, test_accuracies):
    # Plot validation accuracy across epochs for each model
    plt.figure(figsize=(10, 6))
    colors = []
    

### Please enter your solution here ###

    
    plt.grid(True)
    plt.show()

In [ ]:
plot_val_accuracy_history(histories, test_accuracies)

# PyTorch 

Congratulations! 🥳🎉 You've just written a neural network in NumPy from scratch and plotted the validation accuracy across multiple epochs and learning rates! Now it's time to **do it all over again** with the help of the PyTorch (usually and subsequently called Torch) framework. Luckily we won't have to write nearly as many components as a part of our set up process, as Torch already has them built in.

**Fun fact:** You can click on most of the code snippets in the text cells of this section. 

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms

## Defining the computational device

Torch is built to run on the GPU, usually a cuda-enabled one, but it can also run on the CPU if a GPU isn't available. In order to take advantage of a GPU, we'll have to explicitly move data to the GPU in order to utilize these performance benefits. Clearly, it's unknown to me whether your computer has a cuda-enabled GPU, so we'll have to find the correct device to train on. If you don't have a GPU, moving data around might seem redundant, as everything starts on the CPU, but it's good practice and proper form for Torch code.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Creating the Datasets

To instantiate the MNIST dataset we'll be using a built-in dataset from the [`torchvision`](https://docs.pytorch.org/vision/stable/index.html) library. Luckily this makes things easy for us. **Be aware** that it will download the entirety of the MNIST dataset to the same directory where this notebook resides.

Torch, in comparison to NumPy, is a framework, and as such tries to be generalizable and includes a lot of built-in computation in order to do so. In comparison, our NumPy implementation is pretty bare-bones. In order to make our network train in a reasonable amount of time, I've decreased the size of our training set. 

In [ ]:
from torch.utils.data import Subset
import numpy as np

transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0,1] float tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Standard MNIST normalization
])

# Download training and test sets to current directory (i.e., "./")
train_dataset = datasets.MNIST(
    root=".",  # Save data to current directory
    train=True,
    transform=transform,
    download=True
)

train_set_size = len(train_dataset)
indices = torch.arange(train_set_size)
# We'll be using a smaller training set, as noted in the above description
train_idxs = np.random.choice(50000, size=5000, replace=False)
val_idxs = indices[50000:]

val_dataset = Subset(train_dataset, train_idxs)
train_dataset = Subset(train_dataset, train_idxs)

test_dataset = datasets.MNIST(
    root=".",  # Save data to current directory
    train=False,
    transform=transform,
    download=True
)

print("Shape of sample in the torchvision MNIST dataset:", train_dataset[0][0].shape)

## Building the Model

To build the model we've just built in Numpy, we'll need to use the corresponding components in PyTorch, namely [`Flatten`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Flatten.html), [`Linear`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html), [`ReLU`](https://docs.pytorch.org/docs/stable/generated/torch.nn.ReLU.html), and [`Sequential`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html). 

The  [`Sequential`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class is a container in which we will hold and nicely manage all of our layers.  [`Sequential`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html) itself is a subclass of [`Module`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html) which contains more information on the [`Sequential`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html) methods we'll be using in this assignment. 

To make things simple, we'll define a class called `MNISTNet`, make it a subclass of [`Module`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html), and define our neural network in there so it's easier to instantiate a new one whenever we want.

We will only be using this to classify MNIST images, so feel free to hardcode the dimensions into the [`Linear`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html) layers.

Importantly, what is [`Flatten`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Flatten.html)? In the NumPy implementation our samples were represented as a one dimensional array. In the [`torchvision`](https://docs.pytorch.org/vision/stable/index.html) version of the dataset, however, the are three dimensional tensors, as you can see in the output of the above cell. The [`Linear`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer only accepts one dimensional tensors equal to its input size, and as such the input needs to be transformed into one, which is just what [`Flatten`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Flatten.html) does. Essentially, a preprocessing step conveniently baked into our network model.

After defining the network in the `__init__()` method, we will also initialize the network's weights, but the code for that is already written. 

In [ ]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()

### Please enter your solution here ###

        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            # input_size = m.weight.size(1)  # size: [out_features, in_features]
            # scale = (2.0 / input_size) ** 0.5
            nn.init.normal_(m.weight, mean=0.0, 
                            std=(2.0 / m.weight.size(1)) ** 0.5)
            # m.weight.data *= scale  # scale after normal init
            nn.init.constant_(m.bias, 0)
    
    def forward(self, x):

### Please enter your solution here ###



## Preparation for training

In Torch, in order to use our dataset we have to encase it in a fancy iterator called a [`DataLoader`](https://docs.pytorch.org/docs/stable/data.html). It allows us to set the batch size, shuffle our data (or not), among other things, all of which is useful for training. The [`DataLoader`](https://docs.pytorch.org/docs/stable/data.html) class can be found in `torch.utils.data`.

Below do the following:
- Create [`DataLoader`](https://docs.pytorch.org/docs/stable/data.html)s for our train, validation, and test sets. Set the batch size to 64, as is the case in our NumPy implementation. Set `shuffle=True` for the training dataloader and `shuffle=False` for the others.
- Instantiate a [`nn.CrossEntropyLoss`](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) function.

In [ ]:
from torch.utils.data import DataLoader

# Instantiate DataLoaders for the training set, validation set, and test set

### Please enter your solution here ###



## Writing the training routine

We'll break the training routine into two parts. In the first part we'll write a function to train for one epoch. This can be useful, as we don't have to duplicate code when evaluating our model. In the second part we'll build our training routine across multiple epochs, collecting validation accuracy for each epoch as we go.

Beyond dataloaders we'll need two other components:

**Loss Function** -- The loss function has the same purpose as in the NumPy implementation. Sometimes you see the loss function be called a criterion, which is usually used to describe a part of a loss function. We'll instantiate this in the first part.

**Optimizer** -- This is the method we use to update our model weights based on our gradients. Optimizers can be found in [`torch.optim`](https://docs.pytorch.org/docs/stable/optim.html). Here we will be specifying our learning rate when instantiating this. We'll instantiate this in the second part, but use it as an argument in the first part.

For every epoch we will want to keep track of the accuracy and loss.

First we'll do a parameter check. If the model is training, we'll need an optimizer. Make sure that this is the case and throw a `RuntimeError` if it is not.

We will then want to tell our model whether it's training or not so it knows if it should keep track of gradient information. If it is training, we want to set it to train mode, [`model.train()`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train), and if not we want set it to evaluation mode, [`model.eval()`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval). Further we want to find the **context** which will determine whether we will be tracking gradients, and likewise will be set differently depending if we are trianing or not. If we are training the context should be equal to [`torch.enable_grad()`](https://docs.pytorch.org/docs/stable/generated/torch.enable_grad.html#enable-grad) and if we are not it should be [`torch.no_grad()`](https://docs.pytorch.org/docs/stable/generated/torch.no_grad.html#no-grad). We will want to wrap our training loop in a `with` statement that sets this context (i.e. `with context:`). I would recommend setting `context` as a variable before the training loop.

In addition, specify the loss function as [`nn.CrossEntropyLoss()`](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

Next we'll iterate through the dataloader, which returns a tuple at each iteration `(samples, labels)` or `(X, y)`. For every batch we'll want to do the following:
1. If we're training, clear all the gradients tracked by the optimizer. This can be done with [`optimizer.zero_grad()`](https://docs.pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD.zero_grad).
2. Run the samples through the model. As good practice, be sure to transfer the samples to `device`, or rather `X.to(device)`. 
3. Calculate the loss.
4. If you're training, propagate the loss backwards through the network by calling `loss.backward()`. This will compute the gradients. In addition you want to step the optimizer, or move all the weights along their gradients, by calling [`optimizer.step()`](https://docs.pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD.step)
5. Update the total correct, total number of samples encountered in the epoch thus far, and total loss. When calculating the total correct and the loss, you'll likely need to use the `argmax()` method, and the `item()` method.

Finally, calculate and return the accuracy and average loss of the epoch.  

In [ ]:
def run_epoch(model, dataloader, device, optimizer=None, is_train=False):


### Please enter your solution here ###


    
    

Second, we'll define our training routine in terms of epochs, collecting the losses and accuracies of the training and validation sets as we go along. Luckily, this is a bit less involved than the previous function.

1. Move the model to the device (yes again, as a part of good practice)
3. Declare and [`SGD`](https://docs.pytorch.org/docs/stable/generated/torch.optim.SGD.html#sgd) optimizer with the specified learning rate. All other hyperparameters should be zero. You will need to pass the model's parameters, essentially telling the optimizer what it can train, which can be done with the model's `parameters()` method.
4. Iterate through the specified number of epochs, running first on the training set, then on the validation set. Be sure to collect the accuracies and losses along the way. Further, after every epoch, print to the console  the epoch number out of the total number of spcified epochs, the losses of both datasets, and the accuracies of both datasets.
5. Package the losses and accuracies of both data partitions into a dictionary and return this.

The model, as a dynamic object, does not need to be returned. By running the training routine on it, we know that we have modified this object. 

In [ ]:
def train(model, train_dataloader, val_dataloader, device, epochs=20, lr=0.1):


### Please enter your solution here ###

    

## Putting everything together

Like last time, train three models with three different learning rates `0.01`, `0.1`, and `1.0` and plot their validation accuracy and resulting test accuracy. Train each model for `20` epochs. 

Also like last time, if you do this in a loop, which is suggested, be sure to reinstantiate your model every time. 

In [ ]:

### Please enter your solution here ###

